
# Light Forecasting Demo (with Dummy Data)

This notebook generates a realistic synthetic daily time series in the **first cell**, performs exploratory analysis, and runs a few **lightweight forecasting baselines** (naive, seasonal naive, moving average, simple exponential smoothing) plus a **Fourier + trend regression**. Charts use **matplotlib** only.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
rng = np.random.default_rng(42)

# Create daily dates (about 4.5 years ending today)
end = pd.Timestamp.today().normalize()
start = end - pd.Timedelta(days=4*365 + 240)
ds = pd.date_range(start, end, freq='D')

# Build components
n = len(ds)
t = np.arange(n)

# Trend (mild quadratic)
trend = 0.02 * t + 0.00005 * (t**2)

# Weekly seasonality (period 7)
weekly = 3 * np.sin(2 * np.pi * t / 7) + 1.5 * np.cos(2 * np.pi * 2 * t / 7)

# Yearly seasonality (period ~365.25)
yearly = 8 * np.sin(2 * np.pi * t / 365.25) + 4 * np.cos(2 * np.pi * 3 * t / 365.25)

# Random noise
noise = rng.normal(0, 2.5, size=n)

# Occasional promo/event lifts
promo = np.zeros(n)
for _ in range(12):
    start_idx = rng.integers(0, n-14)
    length = rng.integers(3, 10)
    promo[start_idx:start_idx+length] += rng.normal(12, 3)

# Occasional outliers
outliers = np.zeros(n)
outlier_idx = rng.choice(n, size=10, replace=False)
outliers[outlier_idx] = rng.normal(25, 6, size=10) * rng.choice([-1,1], size=10)

# Base level
base = 50

# Final series
y = base + trend + weekly + yearly + promo + outliers + noise

df = pd.DataFrame({'ds': ds, 'y': y})
df = df.set_index('ds').asfreq('D')

print("Data shape:", df.shape)
df.head()


## Quick look at the data
We'll peek at the head/tail and basic statistics.

In [ ]:

display(df.head())
display(df.tail())
display(df.describe())

plt.figure(figsize=(10,4))
plt.plot(df.index, df['y'])
plt.title('Raw Daily Series')
plt.xlabel('Date')
plt.ylabel('y')
plt.grid(True)
plt.show()


## Rolling averages and variability
A quick check of rolling mean and rolling standard deviation.

In [ ]:

roll = df['y'].rolling(30, min_periods=1)
plt.figure(figsize=(10,4))
plt.plot(df.index, df['y'], label='y', alpha=0.6)
plt.plot(df.index, roll.mean(), label='30d mean')
plt.plot(df.index, roll.std(), label='30d std')
plt.title('Rolling 30-Day Mean & Std')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.legend()
plt.show()


## Seasonality views
Daily (day-of-week) and monthly patterns to visualize weekly and annual seasonality.

In [ ]:

dow = df.copy()
dow['dow'] = dow.index.dayofweek
dow_mean = dow.groupby('dow')['y'].mean()

plt.figure(figsize=(8,3))
plt.plot(dow_mean.index, dow_mean.values, marker='o')
plt.xticks(range(7), ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
plt.title('Average by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Average y')
plt.grid(True)
plt.show()

In [ ]:

mo = df.copy()
mo['month'] = mo.index.month
mo_mean = mo.groupby('month')['y'].mean()

plt.figure(figsize=(8,3))
plt.plot(mo_mean.index, mo_mean.values, marker='o')
plt.xticks(range(1,13))
plt.title('Average by Month')
plt.xlabel('Month')
plt.ylabel('Average y')
plt.grid(True)
plt.show()


## Train / Test split
We'll hold out the last 90 days for evaluation.

In [ ]:

horizon = 90
train = df.iloc[:-horizon].copy()
test = df.iloc[-horizon:].copy()
print("Train:", train.shape, " Test:", test.shape)


## Baseline forecasts
We'll implement a few simple baselines:
- **Naive**: Forecast equals the last observed value from the train set
- **Seasonal Naive (weekly)**: Forecast equals the value from 7 days ago
- **Moving Average (7-day)**: Rolling average updates one step ahead

In [ ]:

def mae(y_true, y_pred):
    return float(np.mean(np.abs(y_true - y_pred)))

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def mape(y_true, y_pred):
    eps = 1e-8
    return float(np.mean(np.abs((y_true - y_pred) / (np.maximum(np.abs(y_true), eps)))) * 100)

y_train = train['y']
y_test = test['y']

# Naive
naive_forecast = pd.Series(y_train.iloc[-1], index=test.index)

# Seasonal naive (weekly)
seasonal_lag = 7
seasonal_naive = pd.Series(index=test.index, dtype=float)
for i, ts in enumerate(test.index):
    ref = ts - pd.Timedelta(days=seasonal_lag)
    if ref in df.index:
        seasonal_naive.iloc[i] = df.loc[ref, 'y']
    else:
        seasonal_naive.iloc[i] = y_train.iloc[-1]

print("Naive  - RMSE:", rmse(y_test, naive_forecast), "MAE:", mae(y_test, naive_forecast), "MAPE:", mape(y_test, naive_forecast))
print("S-Naive- RMSE:", rmse(y_test, seasonal_naive), "MAE:", mae(y_test, seasonal_naive), "MAPE:", mape(y_test, seasonal_naive))

In [ ]:

# Moving average (7-day), one-step ahead updating with observed values
window = 7
hist = y_train.copy()
ma_preds = []
for ts in test.index:
    if len(hist) >= window:
        ma_preds.append(hist.iloc[-window:].mean())
    else:
        ma_preds.append(hist.mean())
    # After predicting, "observe" the true test value to update rolling history
    hist = pd.concat([hist, pd.Series([y_test.loc[ts]], index=[ts])])

ma_forecast = pd.Series(ma_preds, index=test.index)

print("MA(7)  - RMSE:", rmse(y_test, ma_forecast), "MAE:", mae(y_test, ma_forecast), "MAPE:", mape(y_test, ma_forecast))


## Simple Exponential Smoothing (SES)
We'll tune \(\alpha\) by grid search (no external libraries) using one-step-ahead test prediction.

In [ ]:

def ses_one_step(train_series, test_series, alpha):
    # initialize level using first train observation
    S = float(train_series.iloc[0])
    # fit on train
    for y in train_series.iloc[1:]:
        S = alpha * float(y) + (1 - alpha) * S
    # one-step ahead predictions across test
    preds = []
    for y in test_series:
        preds.append(S)  # forecast for this step
        S = alpha * float(y) + (1 - alpha) * S  # update after observing true
    return np.array(preds), S  # return final level too

alphas = np.linspace(0.05, 0.95, 19)
best_alpha, best_rmse = None, float('inf')
best_preds, best_S = None, None

for a in alphas:
    preds, S_final = ses_one_step(y_train, y_test, a)
    r = rmse(y_test.values, preds)
    if r < best_rmse:
        best_rmse = r
        best_alpha = a
        best_preds = preds
        best_S = S_final

ses_forecast = pd.Series(best_preds, index=test.index)

print(f"Best alpha: {best_alpha:.2f}")
print("SES     - RMSE:", rmse(y_test, ses_forecast), "MAE:", mae(y_test, ses_forecast), "MAPE:", mape(y_test, ses_forecast))


## Fourier + Trend Regression
We fit a simple linear model (via least squares) with:
- Intercept, linear and quadratic trend
- Weekly Fourier terms (order=3)
- Yearly Fourier terms (order=5)

In [ ]:

def fourier_features(t, period, order):
    X = []
    for k in range(1, order+1):
        X.append(np.sin(2*np.pi*k*t/period))
        X.append(np.cos(2*np.pi*k*t/period))
    return np.column_stack(X) if X else np.zeros((len(t),0))

# Build global time index
t_all = np.arange(len(df))
t_train = np.arange(len(train))
t_test = np.arange(len(train), len(df))

# Design matrix
X_train_parts = [
    np.ones_like(t_train),
    t_train,
    t_train**2,
    fourier_features(t_train, period=7, order=3),
    fourier_features(t_train, period=365.25, order=5)
]
X_train = np.column_stack(X_train_parts)

y_tr = y_train.values
beta, *_ = np.linalg.lstsq(X_train, y_tr, rcond=None)

# Predict on test
X_test_parts = [
    np.ones_like(t_test),
    t_test,
    t_test**2,
    fourier_features(t_test, period=7, order=3),
    fourier_features(t_test, period=365.25, order=5)
]
X_test = np.column_stack(X_test_parts)
ft_forecast = pd.Series(X_test @ beta, index=test.index)

# Metrics
print("Fourier - RMSE:", rmse(y_test, ft_forecast), "MAE:", mae(y_test, ft_forecast), "MAPE:", mape(y_test, ft_forecast))

# Keep residual std for simple CIs later
resid = y_tr - (X_train @ beta)
sigma = float(np.std(resid, ddof=1))
sigma

In [ ]:

metrics = pd.DataFrame({
    'model': ['Naive', 'SeasonalNaive', 'MA(7)', 'SES', 'FourierTrend'],
    'RMSE': [
        rmse(y_test, pd.Series(y_train.iloc[-1], index=test.index)),
        rmse(y_test, pd.Series(seasonal_naive.values, index=test.index)),
        rmse(y_test, ma_forecast),
        rmse(y_test, ses_forecast),
        rmse(y_test, ft_forecast),
    ],
    'MAE': [
        mae(y_test, pd.Series(y_train.iloc[-1], index=test.index)),
        mae(y_test, pd.Series(seasonal_naive.values, index=test.index)),
        mae(y_test, ma_forecast),
        mae(y_test, ses_forecast),
        mae(y_test, ft_forecast),
    ],
    'MAPE': [
        mape(y_test, pd.Series(y_train.iloc[-1], index=test.index)),
        mape(y_test, pd.Series(seasonal_naive.values, index=test.index)),
        mape(y_test, ma_forecast),
        mape(y_test, ses_forecast),
        mape(y_test, ft_forecast),
    ]
}).sort_values('RMSE').reset_index(drop=True)

display(metrics)

plt.figure(figsize=(8,3))
plt.bar(metrics['model'], metrics['RMSE'])
plt.title('RMSE by Model (Lower is Better)')
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.grid(True, axis='y')
plt.show()


## Visualize forecasts on the test window
We'll overlay the test period with a few model predictions.

In [ ]:

plt.figure(figsize=(10,4))
plt.plot(y_test.index, y_test.values, label='Actual')
plt.plot(test.index, seasonal_naive.values, label='Seasonal Naive')
plt.plot(test.index, ma_forecast.values, label='MA(7)')
plt.plot(test.index, ses_forecast.values, label=f'SES (alpha={best_alpha:.2f})')
plt.plot(test.index, ft_forecast.values, label='Fourier+Trend')
plt.title('Test Period: Actual vs Forecasts')
plt.xlabel('Date')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()


## Forecast the next 90 days
We'll pick the **best RMSE** model and produce a 90-day forecast with simple confidence bands where applicable.

In [ ]:

# Choose best by RMSE
best_model = metrics.iloc[0]['model']
print("Best model by RMSE:", best_model)

future_h = 90
last_date = df.index[-1]
future_idx = pd.date_range(last_date + pd.Timedelta(days=1), periods=future_h, freq='D')

if best_model == 'FourierTrend':
    # Build X for future
    t_future = np.arange(len(df), len(df)+future_h)
    X_future_parts = [
        np.ones_like(t_future),
        t_future,
        t_future**2,
        fourier_features(t_future, period=7, order=3),
        fourier_features(t_future, period=365.25, order=5)
    ]
    X_future = np.column_stack(X_future_parts)
    future_pred = X_future @ beta
    # Simple CI from train residual std (not accounting for parameter uncertainty)
    upper = future_pred + 1.96 * sigma
    lower = future_pred - 1.96 * sigma
    future_df = pd.DataFrame({'yhat': future_pred, 'yhat_lower': lower, 'yhat_upper': upper}, index=future_idx)

elif best_model == 'SES':
    # Continue SES forward using last fitted level
    # Re-train SES using best alpha on full df for a stable final level
    preds_full, S_final = ses_one_step(df['y'].iloc[:-1], pd.Series([df['y'].iloc[-1]]), best_alpha)
    level = float(S_final)
    yhat = np.repeat(level, future_h)
    # Use test residual std from SES as simple CI
    ses_resid = y_test.values - ses_forecast.values
    ses_sigma = float(np.std(ses_resid, ddof=1))
    upper = yhat + 1.96 * ses_sigma
    lower = yhat - 1.96 * ses_sigma
    future_df = pd.DataFrame({'yhat': yhat, 'yhat_lower': lower, 'yhat_upper': upper}, index=future_idx)

elif best_model == 'MA(7)':
    # Roll forward using last 7 observed values
    hist = df['y'].copy()
    preds = []
    for _ in range(future_h):
        if len(hist) >= 7:
            preds.append(hist.iloc[-7:].mean())
        else:
            preds.append(hist.mean())
        # simulate observing the prediction (for rolling context); optional
        hist = pd.concat([hist, pd.Series([preds[-1]], index=[hist.index[-1] + pd.Timedelta(days=1)])])
    ma_resid = y_test.values - ma_forecast.values
    s = float(np.std(ma_resid, ddof=1))
    upper = np.array(preds) + 1.96 * s
    lower = np.array(preds) - 1.96 * s
    future_df = pd.DataFrame({'yhat': preds, 'yhat_lower': lower, 'yhat_upper': upper}, index=future_idx)

elif best_model == 'SeasonalNaive':
    # Repeat last week's pattern
    future_vals = []
    hist = df['y'].copy()
    for i in range(future_h):
        ref = hist.index[-7]
        future_vals.append(hist.loc[ref])
        hist = pd.concat([hist, pd.Series([future_vals[-1]], index=[hist.index[-1] + pd.Timedelta(days=1)])])
    sn_resid = y_test.values - seasonal_naive.values
    s = float(np.std(sn_resid, ddof=1))
    upper = np.array(future_vals) + 1.96 * s
    lower = np.array(future_vals) - 1.96 * s
    future_df = pd.DataFrame({'yhat': future_vals, 'yhat_lower': lower, 'yhat_upper': upper}, index=future_idx)

else:  # Naive
    last = df['y'].iloc[-1]
    yhat = np.repeat(last, future_h)
    nv_resid = y_test.values - naive_forecast.values
    s = float(np.std(nv_resid, ddof=1))
    upper = yhat + 1.96 * s
    lower = yhat - 1.96 * s
    future_df = pd.DataFrame({'yhat': yhat, 'yhat_lower': lower, 'yhat_upper': upper}, index=future_idx)

# Plot
plt.figure(figsize=(10,4))
plt.plot(df.index[-180:], df['y'].iloc[-180:], label='History (last 180d)')
plt.plot(future_df.index, future_df['yhat'], label='Forecast')
plt.fill_between(future_df.index, future_df['yhat_lower'], future_df['yhat_upper'], alpha=0.2, label='~95% Interval')
plt.title(f'Next {future_h} Days Forecast — {best_model}')
plt.xlabel('Date')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()
